In [1]:
sc

In [2]:
import json 
import re
import os
import glob

In [3]:
def parsePersons(df):
    personsCollection = open('PersonsCollection.txt', 'a')
    for i in range(len(df)):
        for j in range(len(df[i])):
            if df[i][j][1] == 'persons':
                person = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                personsCollection.write(person+'\n')

def parseSubject(df):
    subjectCollection = open('SubjectCollection.txt', 'a')
    for i in range(len(df)):
        for j in range(len(df[i])):
            if df[i][j][1] == 'subject':
                subject = re.sub(r'[^a-zA-Z]', ' ', df[i][j][3]).lower()
                subjectCollection.write(subject+'\n')

In [4]:
#archive = open('archivejan1.json', 'r')
# os.listdir('~')
# df.show()
# df.printSchema()
cwd = os.getcwd()
print cwd
for i in  glob.glob(cwd+'/archive*'):
    df = spark.read.json(i)
    df = df.select('response.docs.keywords')
    df = df.first()[0]
    parsePersons(df)
    parseSubject(df)

/Users/jamesuttaro/Desktop/Spring 2017/Big Data - 59927/BDM_FinalProject/BDM_FinalProject/datasetNYTimes


In [5]:
#df = df.select('response.docs.keywords')
#df.printSchema()

In [6]:
#df = df.first()[0] # Selects all the entries inside keywwords

In [7]:
print df[0]
print "======================================"
print df[0][0]
print len(df)
print len(df[0])

[Row(isMajor=None, name=u'type_of_material', rank=u'1', value=u'News'), Row(isMajor=None, name=u'persons', rank=u'2', value=u'Abou Zeid, Mahmoud (1987- )'), Row(isMajor=None, name=u'persons', rank=u'3', value=u'Elshamy, Mohammed (1994- )'), Row(isMajor=None, name=u'subject', rank=u'4', value=u'Demonstrations, Protests, and Riots'), Row(isMajor=None, name=u'subject', rank=u'5', value=u'Photography'), Row(isMajor=None, name=u'subject', rank=u'6', value=u'Politics and Government'), Row(isMajor=None, name=u'glocations', rank=u'7', value=u'Egypt')]
Row(isMajor=None, name=u'type_of_material', rank=u'1', value=u'News')
5457
7


### MAP REDUCE ON TEXT FILES

In [11]:
rdd = sc.textFile(cwd+"/PersonsCollection.txt")
print cwd

/Users/jamesuttaro/Desktop/Spring 2017/Big Data - 59927/BDM_FinalProject/BDM_FinalProject/datasetNYTimes


In [13]:
rdd.map(lambda x: (x,1)) \
        .reduceByKey(lambda x,y: x+y) \
        .take(10)

[(u'reider  jonah', 1),
 (u'shannon  christopher', 1),
 (u'litiatco  armando', 2),
 (u'zeid ra ad zeid al hussein', 1),
 (u'magoon  daniel j', 1),
 (u'chozick  amy', 1),
 (u'wells  jane', 2),
 (u'afineevsky  evgeny         ', 2),
 (u'browder  ben', 1),
 (u'garrett  myles         ', 3)]

In [19]:
rdd.take(10)

[u'trump  donald j',
 u'strine  leo e jr',
 u'syndergaard  noah',
 u'montero  rafael',
 u'collins  terry l',
 u'soros  george',
 u'orban  viktor',
 u'de blasio  bill',
 u'campbell  thomas p',
 u'coburn  erin']

In [75]:
import heapq

In [ ]:

# def mapper(line):
#     for word in line:
#         yield(word.lower(),1)

# def reducer(line):
#     yield (word, sum(counts))
    
